Visualisierung im Raum
==============
Dieses Notebook soll am Beispiel der HGB-Daten einen Einblick geben, wie Punktdaten in Python räumlich visualisiert werden können. Für dieses Notebook wird hauptsächlich die Bibliothek "ipyleaflet" benutzt. Die Dokumentation von dieser Bibliothek kann unter folgendem Link aufgerufen werden: https://ipyleaflet.readthedocs.io.

In [ ]:
# Import packages.
import pandas as pd
from pyproj import Transformer
from ipyleaflet import Map, WMSLayer, Circle, Popup, Heatmap, LayerGroup, LegendControl
import ipywidgets as widgets
import time

# Lese Beispieldaten
Basierend auf dem Demo-Notebook 3.0 (https://colab.research.google.com/drive/1_eNm90UzcLxHm44WJt9mj5YBmnwwELYI?usp=sharing) wurden Beispiel-Suchresultate für dieses Notebook erstellt:
- Es wurden folgende Suchparameter benutzt:
    - YEAR_MIN = 1500
    - YEAR_MAX = 1550
    - XPATH_EXP = "./Body//Reference[@normalization='Heiliggeist-Spital']"
- Die Filterfunktion des Demo-Notebooks wurde nicht benutzt.
- Die Suchresultate wurden als Exceltabelle exportiert. Dateiname: "example_search_results.xlsx".

In [ ]:
# Define data source.
URL_DATA = 'https://github.com/history-unibas/FS2024-Research-Seminar-Demo-Notebook/raw/ec5aeaaedbdf4f75427addbafe285ab433694ea0/example_search_results.xlsx'

# Load the data.
result_df = pd.read_excel(URL_DATA)

# Show the data.
display(result_df)

In [ ]:
# Correct the data structure.
result_df['coll_id'] = result_df['coll_id'].apply(lambda x: x[2:-3])
result_df['coll_house'] = result_df['coll_house'].apply(lambda x: x[2:-3])
result_df['coll_x'] = result_df['coll_x'].apply(lambda x: x[2:-3])
result_df['coll_y'] = result_df['coll_y'].apply(lambda x: x[2:-3])
result_df

# Transformiere die Koordinaten in WGS 84

In [ ]:
# Transform the coordinates of the search results.
def transform_coords(x, y):
    transformer = Transformer.from_crs(2056, 4326)
    return transformer.transform(x, y)
result_df['lat'], result_df['lon'] = zip(*result_df.apply(
    lambda row: transform_coords(row['coll_x'], row['coll_y']), axis=1
    ))
result_df

# Definiere eine Grundkarte
Vorgehen zur Verwendung weiterer Grundkarten des Kantons Basel-Stadt:
1. Suche im Geoportal https://map.geo.bs.ch nach der gewünschten Karte (Layer)
2. Suche den Titel der gewünschten Karte in folgender XML-Datei: https://wms.geo.bs.ch/?SERVICE=wms&REQUEST=GetCapabilities
3. Kopiere der "Name" der ausgewählten Karte und ersetze der Parameter "layers" in nachfolgender Codezelle.

In [ ]:
# Define the basemap.
basemap = WMSLayer(
    url='https://wms.geo.bs.ch/',
    layers='HP_Situationsplan_Basel_1862',
    attribution='Geodaten Kanton Basel-Stadt'
)

# Show the basemap on a map.
m1 = Map(basemap=basemap, center=(47.557, 7.595), zoom=14)
m1

# Stelle die Suchresultate als Punkte auf der Karte dar

In [ ]:
# Define a map.
m2 = Map(basemap=basemap, center=(47.557, 7.595), zoom=14)

# Add the search results to the map.
for index, row in result_df.iterrows():
    
    # Create Circle.
    circle = Circle(location=(row['lat'], row['lon']),
                    radius=3,
                    color='blue',
                    fill_color='blue')

    # Add the circle to the map.
    m2.add(circle)

#  Display the map.
m2

# Definiere für die Suchresultate ein Pop-up

In [ ]:
# Define a map.
m3 = Map(basemap=basemap, center=(47.557, 7.595), zoom=14)

# Add the search results to the map.
for index, row in result_df.iterrows():
    
    # Create Circle.
    circle = Circle(location=(row['lat'], row['lon']),
                    radius=3,
                    color='blue',
                    fill_color='blue')
    
    # Add pop-up to display attributes of the circle.
    popup_content = f"""
        ID: {row['coll_id']}<br>
        House: {row['coll_house']}<br>
        Pages: {row['doc_pageNum']}<br>
        Year: {row['doc_date']}<br>
        <a href="{row['doc_image']}">Open Image</a>
        """
    popup = Popup(location=(row['lat'], row['lon']), child=widgets.HTML(popup_content), close_button=True)
    circle.popup = popup

    # Add the circle to the map.
    m3.add(circle)

#  Display the map.
m3

# Suchresultate pro Kategorie darstellen

In [ ]:
# Define the column of the dataframe according to which coloring is to take place.
CATEGORY_COLUMN = 'res_head'

# Define the colors per category.
CATEGORY_COLOR = {'Spital ': 'red',
                  'Spittal ': 'blue',
                  'Spitals ': 'green',
                  'else': 'black'
                  }

# Define a map.
m4 = Map(basemap=basemap, center=(47.557, 7.595), zoom=14)

# Add the search results to the map.
for index, row in result_df.iterrows():
    
    # Determine color based on the value of CATEGORY_COLUMN.
    color = CATEGORY_COLOR.get(row[CATEGORY_COLUMN])
    if color is None:
        color = CATEGORY_COLOR.get('else')
    
    # Create circle using color based on category.
    circle = Circle(location=(row['lat'], row['lon']),
                    radius=3,
                    color=color,
                    fill_color=color)
    
    # Add the circle to the map.
    m4.add(circle)

# Add a legend.
legend = LegendControl(legend=CATEGORY_COLOR)
m4.add(legend)

# Display the map.
m4

# Erstelle eine Heatmap

In [ ]:
# Define a map.
m5 = Map(basemap=basemap, center=(47.557, 7.595), zoom=14)

# Create a heatmap layer.
heatmap_layer = Heatmap(locations=result_df[['lat', 'lon']].values.tolist(),
                        radius=5,
                        blur=8,
                        gradient={0.4: 'blue', 0.6: 'cyan', 0.7: 'lime', 0.8: 'yellow', 1.0: 'red'})

# Add the heatmap layer to the map.
m5.add_layer(heatmap_layer)

# Display the map.
m5

# Animierte Visualisierung über die Zeit

In [ ]:
# Time interval in years.
TIME_INTERVAL = 10

# Time in seconds showing each interval.
TIME_STEP = 5

# Define a map.
m6 = Map(basemap=basemap, center=(47.557, 7.595), zoom=14)

#  Display the map.
display(m6)

# Determine minimal and maximal year.
min_year = result_df['doc_date'].min()
max_year = result_df['doc_date'].max()

# Initialize a group layer.
layer_group = LayerGroup()
m6.add(layer_group)

# Initialize a legend.
legend = LegendControl(legend={})
m6.add(legend)

# Create circles for each time interval.
min_interval = min_year
max_interval = min_interval + TIME_INTERVAL - 1
while min_interval <= max_year:
    # Define the sleeping time.
    time.sleep(TIME_STEP)
    
    # Define a temporary layer.
    layer_group_new = LayerGroup()

    # Determine the results within the current time interval.
    result_interval = result_df[(result_df['doc_date'] >= min_interval) & (result_df['doc_date'] <= max_interval)]

    for index, row in result_interval.iterrows():
        # Create circle.
        circle = Circle(location=(row['lat'], row['lon']),
                        radius=3,
                        color='blue',
                        fill_color='blue')

        # Add circle to the group layer.
        layer_group_new.add(circle)

    # Update the group layer.
    m6.remove(layer_group)
    layer_group = layer_group_new
    m6.add(layer_group)

    # Update the legend.
    m6.remove(legend)
    legend = LegendControl(legend={f'{min_interval} - {min(max_interval, max_year)}': 'blue'})
    m6.add(legend)

    # Update the interval.
    min_interval += TIME_INTERVAL
    max_interval += TIME_INTERVAL

# Exportiere die Karte als html

In [ ]:
m5.save('heatmap.html')